In [1]:
%pip install -q git+https://github.com/Kajachuan/ceia-final-project

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 4.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 818.9/818.9 kB 26.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 51.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.3/121.3 kB 7.7 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [2]:
import torch

from PIL import Image
from torch.utils.data import DataLoader, Dataset
from torchvision.transforms import v2

from ceia_final_project.datasets import ArgentinaSentinel2Dataset
from ceia_final_project.transforms import SegmentationTransform
from ceia_final_project.modules import LightningSegmentation

from lightning.pytorch import Trainer
from lightning.pytorch.loggers import TensorBoardLogger
from lightning.pytorch.callbacks import EarlyStopping, ModelCheckpoint

In [3]:
dataset_root_path = '/kaggle/input/ceia-sentinel'

In [4]:
train_transform = SegmentationTransform(subset='train')
valid_transform = SegmentationTransform(subset='valid')

In [5]:
train_dataset = ArgentinaSentinel2Dataset(dataset_root_path, 'train', train_transform, 2048)
valid_dataset = ArgentinaSentinel2Dataset(dataset_root_path, 'valid', valid_transform, 256)

In [6]:
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=16, shuffle=False)

In [7]:
model_name = 'swinunetr_v2'
encoder_name = ''
loss_name = 'dice'
segmentation = LightningSegmentation(model_name, encoder_name, loss_name)

/usr/local/lib/python3.10/dist-packages/monai/utils/deprecate_utils.py:221: FutureWarning: monai.networks.nets.swin_unetr SwinUNETR.__init__:img_size: Argument `img_size` has been deprecated since version 1.3. It will be removed in version 1.5. The img_size argument is not required anymore and checks on the input size are run during forward().
  warn_deprecated(argname, msg, warning_category)


In [8]:
accelerator = 'gpu' if torch.cuda.is_available() else 'cpu'
output_dir = '/kaggle/working'
checkpoint_callback = ModelCheckpoint(
    monitor='valid_loss', 
    dirpath=f'{output_dir}/checks/{model_name}_{encoder_name}_{loss_name}', 
    save_top_k=1, 
    save_last=True
)
early_stopping_callback = EarlyStopping(
    monitor='valid_loss', 
    min_delta=0.00, 
    patience=10
)
logger = TensorBoardLogger(
    save_dir=output_dir, 
    name='logs', 
    version=f'{model_name}_{encoder_name}_{loss_name}'
)

In [9]:
trainer = Trainer(
    accelerator=accelerator,
    callbacks=[checkpoint_callback, early_stopping_callback],
    fast_dev_run=False,
    log_every_n_steps=64,
    logger=logger,
    min_epochs=1,
    max_epochs=30,
)

INFO: GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO: HPU available: False, using: 0 HPUs


In [10]:
trainer.fit(
    model=segmentation,
    train_dataloaders=train_loader,
    val_dataloaders=valid_loader
)

INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO: 
  | Name          | Type             | Params | Mode 
-----------------------------------------------------------
0 | model         | SwinUNETR        | 7.2 M  | train
1 | criterion     | DiceLoss         | 0      | train
2 | train_metrics | MetricCollection | 0      | train
3 | valid_metrics | MetricCollection | 0      | train
-----------------------------------------------------------
7.2 M     Trainable params
0         Non-trainable params
7.2 M     Total params
28.737    Total estimated model params size (MB)
317       Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=3` in the `DataLoader` to improve performance.
/usr/local/lib/python3.10/dist-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=3` in the `DataLoader` to improve performance.


Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]